# Step 2. Preprocessing CSV file. (df가 원본csv의 16행부터 시작하도록)

In [ ]:
import pandas as pd
from pathlib import Path

In [3]:
data_dir = Path("data")
csv_files = sorted(data_dir.glob("*.csv"))

if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {data_dir.resolve()}")

# 각 CSV 파일을 개별 DataFrame으로 읽어 별도 변수와 컬렉션에 저장
dfs_map = {}   # 파일명(stem) -> DataFrame
dfs_list = []  # index 순서의 리스트

for i, fp in enumerate(csv_files):
    df_i = pd.read_csv(fp)
    dfs_list.append(df_i)
    dfs_map[fp.stem] = df_i
    # 개별 변수로도 접근 가능하게 전역 네임스페이스에 저장 (예: df_0, df_1, ...)
    globals()[f"df_{i}"] = df_i

In [4]:
dfs_map.keys()  # 파일명(stem) 리스트

dict_keys(['1. 의창구_단독다가구(매매)_실거래가_1501-1506', '1. 의창구_단독다가구(매매)_실거래가_1507-1606', '1. 의창구_단독다가구(매매)_실거래가_1607-1706', '1. 의창구_단독다가구(매매)_실거래가_1707-1806', '1. 의창구_단독다가구(매매)_실거래가_1807-1906', '1. 의창구_단독다가구(매매)_실거래가_1907-2006', '2. 성산구_단독다가구(매매)_실거래가_1501-1506', '2. 성산구_단독다가구(매매)_실거래가_1507-1606', '2. 성산구_단독다가구(매매)_실거래가_1607-1706', '2. 성산구_단독다가구(매매)_실거래가_1707-1806', '2. 성산구_단독다가구(매매)_실거래가_1807-1906', '2. 성산구_단독다가구(매매)_실거래가_1907-2006', '3. 마산합포구_단독다가구(매매)_실거래가_1501-1506', '3. 마산합포구_단독다가구(매매)_실거래가_1507-1606', '3. 마산합포구_단독다가구(매매)_실거래가_1607-1706', '3. 마산합포구_단독다가구(매매)_실거래가_1707-1806', '3. 마산합포구_단독다가구(매매)_실거래가_1807-1906', '3. 마산합포구_단독다가구(매매)_실거래가_1907-2006', '4. 마산회원구_단독다가구(매매)_실거래가_1501-1506', '4. 마산회원구_단독다가구(매매)_실거래가_1507-1606', '4. 마산회원구_단독다가구(매매)_실거래가_1607-1706', '4. 마산회원구_단독다가구(매매)_실거래가_1707-1806', '4. 마산회원구_단독다가구(매매)_실거래가_1807-1906', '4. 마산회원구_단독다가구(매매)_실거래가_1907-2006', '5. 진해구_단독다가구(매매)_실거래가_1501-1506', '5. 진해구_단독다가구(매매)_실거래가_1507-1606', '5. 진해구_단독다가구(매매)_실거래가_1607-1706', '5. 진해구_단독다가구(매매)_실거

In [5]:
# 16행을 컬럼명으로, 17행부터 끝까지를 데이터로 하는 데이터프레임 생성
def process_csv_with_header_at_row(df, header_row_idx=15):
    """
    CSV 파일에서 특정 행을 컬럼명으로 하고, 그 다음 행부터를 데이터로 하는 데이터프레임 생성
    
    Args:
        df: 원본 데이터프레임
        header_row_idx: 헤더로 사용할 행의 인덱스 (0부터 시작, 16행이면 15)
    
    Returns:
        새로운 데이터프레임
    """
    # 헤더 행 추출 (16행 = 인덱스 15)
    new_header = df.iloc[header_row_idx].values.tolist()
    
    # 17행부터 끝까지 데이터 추출 (인덱스 16부터)
    data_rows = df.iloc[header_row_idx + 1:].reset_index(drop=True)
    
    # 새로운 컬럼명 설정
    data_rows.columns = new_header
    
    return data_rows


In [6]:
# 첫 번째 파일로 테스트
df_processed = process_csv_with_header_at_row(dfs_list[0])
print(f"원본 데이터프레임 크기: {dfs_list[0].shape}")
print(f"처리된 데이터프레임 크기: {df_processed.shape}")
print("\n컬럼명:")
print(df_processed.columns.tolist())

원본 데이터프레임 크기: (556, 11)
처리된 데이터프레임 크기: (540, 11)

컬럼명:
['시군구', '번지', '주택유형', '도로조건', '연면적(㎡)', '대지면적(㎡)', '계약년월', '계약일', '거래금액(만원)', '건축년도', '도로명']


In [7]:
# 처리된 데이터프레임 확인
print("처리된 데이터프레임 상위 5개 행:")
df_processed.head()

처리된 데이터프레임 상위 5개 행:


,시군구,번지,주택유형,도로조건,연면적(㎡),대지면적(㎡),계약년월,계약일,거래금액(만원),건축년도,도로명
0,경상남도 창원의창구 대산면 가술리,3**,단독,8m미만,115.71,154,201504,28,"13,650",1990,가술길1번길
1,경상남도 창원의창구 대산면 가술리,3**,단독,8m미만,129.92,214,201506,10,"16,300",1995,가술길
2,경상남도 창원의창구 대산면 가술리,5**,단독,12m미만,27.77,228,201504,1,"5,865",1935,가술산남로
3,경상남도 창원의창구 대산면 가술리,1**,단독,8m미만,82.71,275,201504,14,"13,100",1996,진산대로443번길
4,경상남도 창원의창구 대산면 가술리,2**,단독,8m미만,19.8,298,201505,11,"6,500",2004,가술길23번길


## 전처리 완료한 dataframe 생성하기
processed_dfs_list 변수에 저장 완료.

In [8]:
# 모든 CSV 파일에 대해 동일한 처리 적용
processed_dfs_map = {}   # 파일명(stem) -> 처리된 DataFrame
processed_dfs_list = []  # index 순서의 처리된 DataFrame 리스트

for i, (fp, original_df) in enumerate(zip(csv_files, dfs_list)):
    processed_df = process_csv_with_header_at_row(original_df)
    processed_dfs_list.append(processed_df)
    processed_dfs_map[fp.stem] = processed_df
    # 개별 변수로도 접근 가능하게 전역 네임스페이스에 저장 (예: processed_df_0, processed_df_1, ...)
    globals()[f"processed_df_{i}"] = processed_df

print(f"총 {len(processed_dfs_list)}개의 CSV 파일이 처리되었습니다.")
print(f"처리된 파일들: {list(processed_dfs_map.keys())}")

총 30개의 CSV 파일이 처리되었습니다.
처리된 파일들: ['1. 의창구_단독다가구(매매)_실거래가_1501-1506', '1. 의창구_단독다가구(매매)_실거래가_1507-1606', '1. 의창구_단독다가구(매매)_실거래가_1607-1706', '1. 의창구_단독다가구(매매)_실거래가_1707-1806', '1. 의창구_단독다가구(매매)_실거래가_1807-1906', '1. 의창구_단독다가구(매매)_실거래가_1907-2006', '2. 성산구_단독다가구(매매)_실거래가_1501-1506', '2. 성산구_단독다가구(매매)_실거래가_1507-1606', '2. 성산구_단독다가구(매매)_실거래가_1607-1706', '2. 성산구_단독다가구(매매)_실거래가_1707-1806', '2. 성산구_단독다가구(매매)_실거래가_1807-1906', '2. 성산구_단독다가구(매매)_실거래가_1907-2006', '3. 마산합포구_단독다가구(매매)_실거래가_1501-1506', '3. 마산합포구_단독다가구(매매)_실거래가_1507-1606', '3. 마산합포구_단독다가구(매매)_실거래가_1607-1706', '3. 마산합포구_단독다가구(매매)_실거래가_1707-1806', '3. 마산합포구_단독다가구(매매)_실거래가_1807-1906', '3. 마산합포구_단독다가구(매매)_실거래가_1907-2006', '4. 마산회원구_단독다가구(매매)_실거래가_1501-1506', '4. 마산회원구_단독다가구(매매)_실거래가_1507-1606', '4. 마산회원구_단독다가구(매매)_실거래가_1607-1706', '4. 마산회원구_단독다가구(매매)_실거래가_1707-1806', '4. 마산회원구_단독다가구(매매)_실거래가_1807-1906', '4. 마산회원구_단독다가구(매매)_실거래가_1907-2006', '5. 진해구_단독다가구(매매)_실거래가_1501-1506', '5. 진해구_단독다가구(매매)_실거래가_1507-1606', '5. 진해구_단독다가구(매매)_실거래가_1607-1706

In [9]:
# 처리 결과 확인
print("각 처리된 데이터프레임의 크기:")
for i, df in enumerate(processed_dfs_list):
    file_name = csv_files[i].stem
    print(f"{i}: {file_name} - {df.shape}")

# 각 처리된 데이터프레임의 컬럼 및 샘플 출력
for i, df in enumerate(processed_dfs_list):
    file_name = csv_files[i].stem
    print(f"\n[{i}] {file_name} - 컬럼:")
    print(df.columns.tolist())

    print(f"\n[{i}] {file_name} - 샘플 상위 3개 행:")
    print(df.head(3))

각 처리된 데이터프레임의 크기:
0: 1. 의창구_단독다가구(매매)_실거래가_1501-1506 - (540, 11)
1: 1. 의창구_단독다가구(매매)_실거래가_1507-1606 - (824, 11)
2: 1. 의창구_단독다가구(매매)_실거래가_1607-1706 - (529, 11)
3: 1. 의창구_단독다가구(매매)_실거래가_1707-1806 - (346, 11)
4: 1. 의창구_단독다가구(매매)_실거래가_1807-1906 - (280, 11)
5: 1. 의창구_단독다가구(매매)_실거래가_1907-2006 - (453, 11)
6: 2. 성산구_단독다가구(매매)_실거래가_1501-1506 - (155, 11)
7: 2. 성산구_단독다가구(매매)_실거래가_1507-1606 - (193, 11)
8: 2. 성산구_단독다가구(매매)_실거래가_1607-1706 - (134, 11)
9: 2. 성산구_단독다가구(매매)_실거래가_1707-1806 - (74, 11)
10: 2. 성산구_단독다가구(매매)_실거래가_1807-1906 - (66, 11)
11: 2. 성산구_단독다가구(매매)_실거래가_1907-2006 - (93, 11)
12: 3. 마산합포구_단독다가구(매매)_실거래가_1501-1506 - (380, 11)
13: 3. 마산합포구_단독다가구(매매)_실거래가_1507-1606 - (607, 11)
14: 3. 마산합포구_단독다가구(매매)_실거래가_1607-1706 - (565, 11)
15: 3. 마산합포구_단독다가구(매매)_실거래가_1707-1806 - (409, 11)
16: 3. 마산합포구_단독다가구(매매)_실거래가_1807-1906 - (248, 11)
17: 3. 마산합포구_단독다가구(매매)_실거래가_1907-2006 - (248, 11)
18: 4. 마산회원구_단독다가구(매매)_실거래가_1501-1506 - (483, 11)
19: 4. 마산회원구_단독다가구(매매)_실거래가_1507-1606 - (700, 11)
20: 4. 마산회원구_단독다가구(

In [11]:
# processed_dfs_list 변수에 전처리 완료한 데이터프레임들을 utf-8 csv로 저장
output_dir = Path("processed_data")
output_dir.mkdir(exist_ok=True)

for i, df in enumerate(processed_dfs_list):
    file_name = csv_files[i].stem
    df.to_csv(output_dir / f"{file_name}_processed.csv", index=False, encoding="utf-8")